In [13]:
!pip install  fitz -qq
!pip install  PyMuPDF -qq
!pip install pytesseract opencv-python pillow numpy -qq
!apt-get install tesseract-ocr -qq
!apt-get install tesseract-ocr-kor -qq
!pip install tqdm --qq

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import fitz  # PyMuPDF
import cv2
import pytesseract
from PIL import Image
import numpy as np


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --qq


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
from tqdm import tqdm
import nltk

def summarize_text(text, model, tokenizer):
    # 전체 진행 상태를 표시하기 위한 tqdm 바 생성
    steps = 3  # 전체 3단계 (토크나이징, 생성, 디코딩)
    with tqdm(total=steps, desc="Overall Progress", ncols=100) as pbar:
        # 토크나이징
        prefix = "summarize: "
        inputs = [prefix + text]
        inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
        pbar.update(1)  # 첫 번째 단계 완료 후 진행률 업데이트

        # 생성
        output = model.generate(**inputs, num_beams=3, do_sample=True, min_length=10, max_length=64)
        pbar.update(1)  # 두 번째 단계 완료 후 진행률 업데이트

        # 디코딩
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        result = nltk.sent_tokenize(decoded_output.strip())[0]
        pbar.update(1)  # 세 번째 단계 완료 후 진행률 업데이트

    return result


# pdf - > text 추출
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    content = ''
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        content += page.get_text()
    return content

# 이미지 -> 텍스트 추출
# 이미지 로드
def preprocess_image(image_path):
    # 이미지를 흑백으로 변환
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 이진화: 텍스트를 명확하게 하기 위해 흑백으로 변환
    _, thresh_img = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)

    # 잡음 제거: 가우시안 블러 사용
    blurred_img = cv2.GaussianBlur(thresh_img, (5, 5), 0)

    return blurred_img

# 이미지에서 텍스트 추출
def extract_text(image_path):
    # 전처리된 이미지
    preprocessed_image = preprocess_image(image_path)

    # 텍스트 추출 (한글 언어 데이터 사용)
    text = pytesseract.image_to_string(preprocessed_image, lang='kor')
    return text

img 파일 -> 텍스트

In [4]:
print(extract_text('/content/캡처.PNG'))

이들은 “계엄이 야기한 문제를 고작 '국민들께 풀안과 불면을 끼친 것'으로 묘사한다. 국정 최
고 책임자로서의 엄중한 책임감이라고는 전혀 찾아볼 수 없다. 어떻게 해서든 탄핵을 막아보
겠다는 것이다. 언제든 내란을 획책할 수 있는 기득권 정치세력의 정국 주도권을 끝까지 유
지하겠다는 것"이라고 일침했다.

이들은 "어불성설이다. 국민들은 더 이상 고양이에게 생선 가게를 맡길 생각이 없다. 국민의
힘은 탄핵하라는 국민의 명령을 이행하고 내란을 획책한 자들에 대한 명백한 처벌의 길을 열
어야 한다"며 “국민과 역사 앞에 진정으로 사죄하고 진정 책임지는 일"이라고 역설했다.

5-18민주화운동 공법 3단제(유족회-부상자회-공로자회)와 518기념재단도 성명을 내고 윤
대동령을 향해 "내란을 일으킨 신군부 세력의 폭압적 행위를 용호하고 그들을 정당화하려는
태도를 보였다"며 “5718로 희생된 수많은 국민의 피와 눈물을 모욕하는 행위"라고 비딴했다.

이들 단체는 “국회는 오늘 대한민국 민주주의 근간을 흔드는 반란군에게 다시 권력을 허락
할 것인지 심판할 것인지 션택해야 현다“며 "국민의힘이 국민의 목소리를 외면하고 내란수
괴의 하수인이 될지 국민 앞에 무름 꿀고 사죄하는 올바른 선택을 할지 국민들이 지켜볼
것"이라고 강조했다.

윤석열 정권퇴진 광주비상행동은 탄핵 소추안 의결이 예정된 오후 5시 광주 동구 578민주
광장에서 4자 시민종결기대회를 연다. 지역시민 800여명은 상경해 국회 양 탄핵 촉구 집회
에 힘을 보탠다.



PDF에서 텍스트 추출

In [7]:
print(extract_text_from_pdf('/content/프로젝트_보고서.pdf'))

프로젝트보고서
Markdown 및AI 프롬프트활용학습
프로젝트개요
이프로젝트는Markdown을사용하여문서를
작성하고, 프롬프트를활용하여정확한정보나
코드를요청하는방법을학습하는것을목표로
합니다.
Markdown을통해문서화작업을효율적으로
진행하며, Python 코드예시를통해프로그래
밍기술을적용하고,
AI 프롬프트를활용하여시스템과상호작용하
는방법을익힙니다.
프로젝트목표
1. Markdown 기본문법배우기및적용하기
2. 프롬프트를활용하여AI 또는프로그램에
효율적으로요청하기
사용된기술
- Markdown: 문서작성및구조화
- Python: 코드예시작성및실행
- AI 프롬프트: AI 시스템과의상호작용
코드예시
# python에서리스트에서짝수만출력하는
코드예시
number = [1, 2, 3, 4, 5]
even_numbers = [num for num in number if 
num % 2 == 0]
print(even_numbers)
코드설명
1. 'number' 리스트에서숫자들을필터링하
여짝수만추출합니다.
2. '리스트컴프리헨션'을사용하여짝수조
건을만족하는숫자들만선택합니다.
3. 짝수만필터링한결과를'even_numbers' 
리스트에저장하고출력합니다.
프로젝트과정
1. Markdown 문법학습및적용
2. 프롬프트작성및AI 상호작용
3. 코드예시작성및결과분석
결론
이프로젝트를통해Markdown을사용한문
서작성방법과Python 코드를문서에통합
하는방법을배웠습니다.
또한, 프롬프트를활용한AI와의상호작용
을통해효율적으로문제를해결하는기술
을익혔습니다.
향후개선사항
- Markdown 문서작성에서다양한스타일
링기능을더활용하여문서를풍성하게만
들기
- AI 프롬프트작성시다양한상황을고려한
프롬프트작성법추가



요약하기

In [10]:
#  한번만 실행하기
model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

In [17]:
txt = '''
프로젝트보고서
Markdown 및AI 프롬프트활용학습
프로젝트개요
이프로젝트는Markdown을사용하여문서를
작성하고, 프롬프트를활용하여정확한정보나
코드를요청하는방법을학습하는것을목표로
합니다.
Markdown을통해문서화작업을효율적으로
진행하며, Python 코드예시를통해프로그래
밍기술을적용하고,
AI 프롬프트를활용하여시스템과상호작용하
는방법을익힙니다.
프로젝트목표
1. Markdown 기본문법배우기및적용하기
2. 프롬프트를활용하여AI 또는프로그램에
효율적으로요청하기
사용된기술
- Markdown: 문서작성및구조화
- Python: 코드예시작성및실행
- AI 프롬프트: AI 시스템과의상호작용
코드예시
# python에서리스트에서짝수만출력하는
코드예시
number = [1, 2, 3, 4, 5]
even_numbers = [num for num in number if
num % 2 == 0]
print(even_numbers)
코드설명
1. 'number' 리스트에서숫자들을필터링하
여짝수만추출합니다.
2. '리스트컴프리헨션'을사용하여짝수조
건을만족하는숫자들만선택합니다.
3. 짝수만필터링한결과를'even_numbers'
리스트에저장하고출력합니다.
프로젝트과정
1. Markdown 문법학습및적용
2. 프롬프트작성및AI 상호작용
3. 코드예시작성및결과분석
결론
이프로젝트를통해Markdown을사용한문
서작성방법과Python 코드를문서에통합
하는방법을배웠습니다.
또한, 프롬프트를활용한AI와의상호작용
을통해효율적으로문제를해결하는기술
을익혔습니다.
향후개선사항
- Markdown 문서작성에서다양한스타일
링기능을더활용하여문서를풍성하게만
들기
- AI 프롬프트작성시다양한상황을고려한
프롬프트작성법추가

'''
summarize_text(txt,model,tokenizer)

Overall Progress: 100%|███████████████████████████████████████████████| 3/3 [00:12<00:00,  4.09s/it]


'프로젝트보고서는 Markdown을 사용하여 문서를 작성하고 프롬프트를 활용하여 정확한 정보나 코드를 요청하는 방법을 학습하는 것을 목표로 한다.'